dataset code: A301

eligibility:
1. after 2016
2. at least 2 admissions
3. admission is eligible if there is at least 1 recognized ccs code
5. episodes can't have multiple diagnostics assigned with more than 1 day delay between each other

process:
1. ccs codes
2. 12 months target window
3. saves delta_days and date of last admission before prediction_period of all datapoints

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [59]:
import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_columns = None
pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', None)

import numpy as np
import os

from datetime import timedelta
import json

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from ICDMappings import ICDMappings
icdmap = ICDMappings()

# Parameters

In [3]:
#where to save the resulting dataset
dataset_filepath = 'data/model_ready_dataset/icare2021_diag_A301/dataset.json'
raw_data_filepath = 'data/model_ready_dataset/icare2021_diag_A301/dataset.csv'

#checks

assert os.path.dirname(dataset_filepath) == os.path.dirname(raw_data_filepath), 'make sure both dataset are saved under the same directory'
assert os.path.isdir(os.path.dirname(dataset_filepath)),'Please create the directory first or try another path to save'

assert not os.path.isfile(dataset_filepath) or not os.path.isfile(raw_data_filepath), 'File exists, are you sure you want to overwrite it? If so, comment this line and run the notebook again'

----

In [4]:
class Icare:
    def __init__(self,data_folder):
        self.data_folder = data_folder
        self.diagnoses_path = 'LS_ANALYTICS.ICARE_CLINICO_DIAGNOSTICOS/index.csv'
        self.atividade_path = 'LS_ANALYTICS.ICARE_ATIVIDADE_HOSPITALAR/index.csv'
        
    
    def _read_diagnoses(self):
        print('Reading diagnostics table...')
        diagnoses_file = os.path.join(self.data_folder,self.diagnoses_path)
        
        df = pd.read_csv(diagnoses_file,sep='\t')
        
        print('begining: ',round(df.memory_usage(index=True).sum() / 1_000_000,1),'Mb') #Mbytes
        
        df = df.drop(columns='UNIDADE')
        #print('drop UNIDADE:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df = df.drop(columns='DATA_FIM')
        #print('drop date_end:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df.loc[:,'PRIORIDADE_DIAGNOSTICO'] = df.PRIORIDADE_DIAGNOSTICO.astype('category')
        #print('PRIORIDADE_DIAGNOSTICO to category:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df = df.drop(columns=['ICD9_DESCRICAO'])
        #print('drop icd9_descricao:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df.loc[:,'DIAGNOSTICO_PRINCIPAL'] = df.DIAGNOSTICO_PRINCIPAL.map({'S':True,'N':False})        
        #print('DIAGNOSTICO_PRINCIPAL to boolean:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        # a single row with the year 9064 ruins pd.to_datetime (overflow since pandas uses miliseconds in dates)
        # tldr:  we will remove that row
        nrows_before = df.shape[0]
        df = df.drop(df.DATA_INICIO.apply(lambda x: x[:4] if x is not np.nan else x).astype(float).where(lambda x: x==9064.0).dropna().index[0])
        nrows_after = df.shape[0]
        assert nrows_before == nrows_after + 1, 'Ooops, expecting to drop exactly 1 row. maybe dataset changed.'
        
        df.loc[:,'DATA_INICIO'] = pd.to_datetime(df.DATA_INICIO,format='%Y-%m-%d %H:%M:%S')
        #print('DATA_INICIO to datetime:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        df = df.drop_duplicates()
        #print('drop duplicates:',df.memory_usage(index=True).sum() / 1_000_000) #Mbytes
        
        # last row is trash
        df = df.iloc[:-1]
        
        print('end: ',round(df.memory_usage(index=True).sum() / 1_000_000),'Mb') #Mbytes
        print('Done')
        return df
        

In [5]:
icare = Icare('../../icare-dataset_2021-08')

In [6]:
df = icare._read_diagnoses()
df.head(1)

Reading diagnostics table...


/tmp/ipykernel_23161/365396413.py:12: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(diagnoses_file,sep='\t')


begining:  178.9 Mb
end:  117 Mb
Done


,EPISODIO,NHC,PRIORIDADE_DIAGNOSTICO,ICD9,DIAGNOSTICO_PRINCIPAL,DATA_INICIO
0,9B91F36DDC2E5D45,7E0717CE88F09A1EB9B7CB48F3520501,ASAP,2578,True,2016-10-19 17:40:00


In [7]:
res = df.groupby(['EPISODIO','NHC']).DATA_INICIO.agg([min,max])
res.head(3)

,,min,max
EPISODIO,NHC,,
000016F136985F31,9D73829E86FD907A3B2DBCB658D43107,2018-05-29 11:00:00,2018-05-29 11:00:00
00002A60B8C0E69D33E22BC4C4938ACB,E34C3A6E5A9DFDCC61EFFE9C3E7BF5D7,2020-10-19 23:47:00,2020-10-19 23:47:00
00003451D06E035DCC1AF5B2F032B4FA,8FFF5F2B2CFDE2CAE785CCD06B921A5E,2021-05-07 17:38:00,2021-05-07 17:38:00


In [8]:
res.head(5)

,,min,max
EPISODIO,NHC,,
000016F136985F31,9D73829E86FD907A3B2DBCB658D43107,2018-05-29 11:00:00,2018-05-29 11:00:00
00002A60B8C0E69D33E22BC4C4938ACB,E34C3A6E5A9DFDCC61EFFE9C3E7BF5D7,2020-10-19 23:47:00,2020-10-19 23:47:00
00003451D06E035DCC1AF5B2F032B4FA,8FFF5F2B2CFDE2CAE785CCD06B921A5E,2021-05-07 17:38:00,2021-05-07 17:38:00
00003E13B7BB0BDC,0E07DB87000F238F62B76F6D4A3C2008,2017-07-10 19:22:00,2017-07-10 19:22:00
000041F59C9A77B5FAB8272B6D539637,DD4192C4ABE03BED93911DA36EB8FB34,2019-11-14 13:10:00,2019-11-14 13:10:00


----

# ICD9 to ICD9_3 then CCS

In [9]:
df.loc[:,'ICD9_3'] = df.ICD9.apply(lambda x: x[:3] if x is not np.nan else x)
df.loc[:,'ICD9_3->CCS'] = icdmap.lookup('icd9_3toccs',df['ICD9_3'])

## define eligibility criteria

1. data after 2016
1. patients with at least 2 admissions
1. all admissions must have at least 1 ccs diagnostic that is eligible

## define windows

3,6,12 months

what metrics to keep track of:
1. distribution of #admissions per target window
1. distribution of # diagnoses per target window
1. distribution of # admissions of input per target window
1. distribution # diagnoses of input per target window

In [ ]:
# eligibility
df = icare._read_diagnoses()

print('Preparing eligibility filtering...')
df['ICD9_3'] = df.ICD9.apply(lambda x: x[:3] if x is not np.nan else x)
df['ICD9_3->CCS'] = icdmap.lookup('icd9_3toccs',df['ICD9_3'])

## after 2016
df = df.loc[df.DATA_INICIO > '2016-01-01']

## admission is eligible if there is at least 1 recognized ccs code
df['is_ccs_na'] = df['ICD9_3->CCS'].isna()
admissions_without_any_eligible_ccs = df.groupby('EPISODIO')['is_ccs_na'].all().where(lambda x: x == True).dropna().index
df = df.loc[~df.EPISODIO.isin(admissions_without_any_eligible_ccs)]

## drop rows where diagnostic is not recognized
df = df.loc[~df['ICD9_3->CCS'].isna()]

## patient with at least 2 admissions
patients_2_admissions = df.groupby('NHC').EPISODIO.size().where(lambda x: x > 1).dropna().index
df = df.loc[df.NHC.isin(patients_2_admissions)]

## episodes with multiple diagnostics can't have diagnostics assigned with more than 1 day delay between each other
episodes_far_diagnostics = df.groupby('EPISODIO').DATA_INICIO.agg([min,max]).diff(axis=1)['max'].dt.days.where(lambda x: x > 0).dropna().index
df = df.loc[~df.EPISODIO.isin(episodes_far_diagnostics)]
print('Done')

df = df.sort_values('DATA_INICIO')
df.DATA_INICIO = pd.to_datetime(df.DATA_INICIO.dt.date,format='%Y-%m-%d')

df.shape
df.EPISODIO.nunique()
df.NHC.nunique()
df.groupby('NHC').EPISODIO.size().value_counts().rename('Distribution of #episodes per eligible patient')

Reading diagnostics table...


/tmp/ipykernel_23161/365396413.py:12: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(diagnoses_file,sep='\t')


begining:  178.9 Mb
end:  117 Mb
Done
Preparing eligibility filtering...


/tmp/ipykernel_23161/2163803607.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_ccs_na'] = df['ICD9_3->CCS'].isna()


Done


(2052106, 9)

In [ ]:
df.loc[:,'ccs_list'] = df['ICD9_3->CCS'].apply(lambda x: [x])
df_ = df.groupby(['NHC','DATA_INICIO'])[['ccs_list']].sum()

In [ ]:
test = df_.copy()

#### 1000 rows
1. 1.68
2. 1.08

#### 10_000 rows
1. 14.4 (8.5x)
2. 8.49 (7.9x)

#### 100_000 rows
1. 189 (112x) (13x)
2. 85 (78x) (10x)

In [ ]:
%%time
#assert 1==2,'Prevent myself from running this cell
m = 12 #months
res = (test
 .groupby('NHC')
 .apply(lambda subdf: 
        subdf.assign(target = 
                     subdf.apply(lambda row: 
                                 subdf
                                 .loc[idx[:,
                                         row.name[1]+timedelta(days=1):row.name[1]+timedelta(days=30*m)
                                         ],
                                      'ccs_list'
                                     ]
                                 .sum(),
                                 axis=1
                                ),
                     history = 
                     subdf.apply(lambda row:
                                 subdf
                                 .loc[idx[:,
                                          :row.name[1]+timedelta(days=1)
                                         ],
                                      'ccs_list'
                                     ]
                                 .tolist(),
                                 axis=1
                                ),
                    )
       )
      )

# Add feature: delta days

In [ ]:
res['delta_days'] = res.reset_index().groupby('NHC')['DATA_INICIO'].diff().dt.days.fillna(0).values

In [72]:
res.loc[:,'target2'] = res.target.apply(lambda x: x if type(x) == list else [])

# Print out some distributions

In [31]:
print(f'Number of non-empty targets out of {res.shape[0]}: {res[res.target != 0].shape[0]}')
print(f'Distribution of target size')
pd.concat([res.loc[res.target != 0,'target'].apply(len).value_counts(normalize=False).iloc[:15],
           res.loc[res.target != 0,'target'].apply(len).value_counts(normalize=True).iloc[:15]
          ],axis=1)

Number of non-empty targets out of 1812829: 1398651
Distribution of target size


,target,target
1,329099,0.235297
2,236673,0.169215
3,174556,0.124803
4,133687,0.095583
5,102097,0.072997
6,80984,0.057902
7,63287,0.045249
8,51172,0.036587
9,40953,0.029280
10,33251,0.023774


In [39]:
res.loc[res.target != 0].groupby('NHC').delta_days.agg('median').describe()[['25%','50%','75%']].rename('Quartiles of all patients of the median delta_days of each patient, on eligible datapoints')

25%     0.5
50%    24.0
75%    68.5
Name: Quartiles of all patients of the median delta_days of each patient, on eligible datapoints, dtype: float64

In [89]:
res2 = res.reset_index(1).rename(columns={'DATA_INICIO':'DATA'})

In [90]:
res2.groupby('NHC').size().describe()[['25%','50%','75%']].rename('Quartiles #admissions eligible per patient')

25%    2.0
50%    4.0
75%    8.0
Name: Quartiles #admissions eligible per patient, dtype: float64

In [91]:
print(f'{res2.index.get_level_values(0).nunique()} patients eligible out of {test.index.get_level_values(0).nunique()}')

262811 patients eligible out of 262811


----

# Save

In [112]:
res2[res2.index.get_level_values(0).isin(res2.groupby('NHC').size().where(lambda x: x == 1).dropna().index)].head(10)

,DATA,ccs_list,target,history,delta_days,target2
NHC,,,,,,
000CC6B6978241BF7EC26026D203B9D7,2016-09-19,"[663.0, 251.0]",0,"[[663.0, 251.0]]",0.0,[]
000EFD57732A014F89170BEF38ABB731,2018-12-18,"[25.0, 50.0, 99.0]",0,"[[25.0, 50.0, 99.0]]",0.0,[]
0024809E1E3E338E55ED20B549AB1EE9,2019-02-06,"[120.0, 238.0]",0,"[[120.0, 238.0]]",0.0,[]
002E33C4113944B16851E7BCE219B94B,2020-04-23,"[670.0, 251.0]",0,"[[670.0, 251.0]]",0.0,[]
003A4D0FE843EBE9AA7C0E15442B8416,2018-02-22,"[122.0, 159.0]",0,"[[122.0, 159.0]]",0.0,[]
003DB1A25AE80675B379FF0C42029AE3,2017-08-21,"[134.0, 99.0]",0,"[[134.0, 99.0]]",0.0,[]
0046B15F18F1434BACC78EB51A3AD6EC,2017-02-17,"[117.0, 663.0]",0,"[[117.0, 663.0]]",0.0,[]
005C74FBD728D5563D3E8B6EE6F2C90D,2016-01-27,"[149.0, 149.0]",0,"[[149.0, 149.0]]",0.0,[]
005DA06D9C44632D05B39575AC632EE3,2020-08-05,"[258.0, 89.0]",0,"[[258.0, 89.0]]",0.0,[]


csv

In [125]:
res2.loc['0000676389D1EE60EB48AF5693F3F3DE']

,DATA,ccs_list,target,history,delta_days,target2
NHC,,,,,,
0000676389D1EE60EB48AF5693F3F3DE,2016-02-28,[670.0],[670.0],[[670.0]],0.0,[670.0]
0000676389D1EE60EB48AF5693F3F3DE,2016-08-08,[670.0],0,"[[670.0], [670.0]]",162.0,[]


In [132]:
res2.loc['50AE695937A38D390DDA2C4FC574D594']

DATA                2021-01-20
ccs_list        [660.0, 155.0]
target                       0
history       [[660.0, 155.0]]
delta_days                 0.0
target2                     []
Name: 50AE695937A38D390DDA2C4FC574D594, dtype: object

In [131]:
a = [res2.loc['50AE695937A38D390DDA2C4FC574D594'].history]
a
for e in a:
    e

[[[660.0, 155.0]]]

[[660.0, 155.0]]

In [92]:
res2 = res2.astype({'DATA':str}) # so that json can save this
res2.to_csv(raw_data_filepath,index=True)

and dict

In [133]:
# where it all begins
data = {}

patients = res2.index.unique()
for idx,p in tqdm(enumerate(patients)):
    
    p_data = res2.loc[p]
    
    history = p_data['history']
    targets = p_data['target2']
    delta_days = p_data['delta_days']
    date_last_history = p_data['DATA']
    
    history = history if type(history) != pd.Series else history.iloc[-1]
    targets = [targets] if type(targets) != pd.Series else targets.tolist()
    delta_days = [delta_days] if type(delta_days) != pd.Series else delta_days.tolist()
    date_last_history = [date_last_history] if type(date_last_history) != pd.Series else date_last_history.tolist()
    
    data[p] = { 'ccs': #only ccs for now
               {
                   'history':history,
                   'targets':targets,
                   'extra_features':
                   {
                       'delta_days': delta_days,
                       'date_last_history': date_last_history
                   }
               }
              }

0it [00:00, ?it/s]

In [134]:
with open(dataset_filepath, 'w') as fp:
    json.dump(data, fp)

----

# test

In [ ]:
with open(dataset_filepath, 'r') as fp:
    test_data = json.load(fp)

In [ ]:
test_data['0000676389D1EE60EB48AF5693F3F3DE']['ccs']